In [1]:
import tensorflow as tf
import keras
import numpy as np
from keras.layers import Dense, Concatenate

In [2]:
# Subclass method to create a model

class TicketClassifier(keras.Model):
    def __init__(self):
        super().__init__()
        self.concatenate_layer = Concatenate() # Concatenation layer
        self.hidden_layer = Dense(units=64,activation="relu") # Hidden layer
        self.priority_score_layer = Dense(units=1,activation="sigmoid") # Priority score layer
        self.department_classifier_layer = Dense(units=4,activation="softmax") # Department classifier layer
        self.difficulty_classifier_layer = Dense(units=3,activation="softmax") # Difficulty classifier layer

    def __call__(self, inputs,training=False): # Input will be passed as a dictionary with these key values ( Training parameter is set to true during training and false during inference)
        # Always include training parameter if we want to train our model
        title_input = inputs["title_data"] 
        body_input = inputs["body_data"]
        tags_input = inputs["tags_data"]

        # The training parameter is used in the output layers to determine whether the model is being used for training or for inference 
        input_concatenated = self.concatenate_layer([title_input, body_input, tags_input]) # Output of the concatenation layer
        hidden_layer_output = self.hidden_layer(input_concatenated) # Output of the concatenation layer passed in hidden layer and output calculated for the hidden layer
        priority_score_output = self.priority_score_layer(hidden_layer_output,training=training) # Output of the hidden layer passed in priority score layer and output calculated
        department_classifier_output = self.department_classifier_layer(hidden_layer_output,training=training) # Output of the hidden layer passed in department classifier layer and output calculated
        difficulty_classifier_output = self.difficulty_classifier_layer(hidden_layer_output,training=training) # Output of the difficulty layer passed in difficulty layer and output calculated

        return priority_score_output, department_classifier_output, difficulty_classifier_output # The outputs of the final output layers are returned

In [3]:
model = TicketClassifier()

In [4]:
# Check the layer names of the model
for layer in model.layers:
    print(layer.name)

concatenate
dense
dense_1
dense_2
dense_3


In [5]:
# If we wanted to decide the output layer units using the input data...like num_departments and num_difficulty classes, we can define them in __init__ like below

#class TicketClassifier(keras.Model):

#def __init__(self,num_departments,num_difficulties): # The variables are mentioned here
#        super().__init__()
#        self.concatenate_layer = Concatenate() 
#        self.hidden_layer = Dense(units=64,activation="relu") 
#        self.priority_score_layer = Dense(units=1,activation="sigmoid") 
#        self.department_classifier_layer = Dense(units=num_departments,activation="softmax")  # The variables are used here
#        self.difficulty_classifier_layer = Dense(units=num_difficulties,activation="softmax") # The variables are used here

#    def __call__(self, inputs): 
#        title_input = inputs["title_data"] 
#        body_input = inputs["body_data"]
#        tags_input = inputs["tags_data"]

#        input_concatenated = self.concatenate_layer([title_input, body_input, tags_input])
#        hidden_layer_output = self.hidden_layer(input_concatenated) 
#        priority_score_output = self.priority_score_layer(hidden_layer_output) 
#        department_classifier_output = self.department_classifier_layer(hidden_layer_output) 
#        difficulty_classifier_output = self.difficulty_classifier_layer(hidden_layer_output) 

#        return priority_score_output, department_classifier_output, difficulty_classifier_output 

In [6]:
# We would then have instantiated the model like below:

# model = TicketClassifier(num_departments=4,num_difficulties=3)

In [7]:
import numpy as np
num_samples = 1500
vocabulary_size =10000
num_tags = 100
department_num = 4
difficulty_num = 3

title_data = np.random.randint(0,2,size=(num_samples,vocabulary_size)) # lowest value , highest value , size
body_data = np.random.randint(0,2,size=(num_samples,vocabulary_size))
tags_data = np.random.randint(0,2,size=(num_samples,num_tags))

priority_data = np.random.random(size=(num_samples,1))
department_data = np.random.randint(0,2,size=(num_samples,department_num))
difficulty_data = np.random.randint(0,2,size=(num_samples,difficulty_num))

In [8]:
# Treaining is set to True, because we are going to train the model
priority_score , department_class , difficulty_class = model({"title_data":title_data,"body_data":body_data,"tags_data":tags_data},training=True) # Dictionary passed with the key values mentioned in the class model and the input_data we created are the values

In [9]:
# Compiling and training the model similar to earlier methods

model.compile(optimizer="rmsprop",
              loss=['mean_squared_error', 'categorical_crossentropy', 'categorical_crossentropy'],
              metrics={'output_1': 'accuracy', 
                                          'output_2': 'accuracy', 
                                          'output_3': 'accuracy'}) # If we want to apply similar metrics or similar losses to 2 or more outputs and need to define a dictionary,
# use output_1 , output_2 ,3 .... --> not able to rename them so use these itself

In [10]:
model.fit({"title_data": title_data, "body_data": body_data, "tags_data": tags_data},
          [priority_data,department_data,difficulty_data],
          epochs=10,
          validation_split=0.2)


Epoch 1/10
38/38 [==============================] - 2s 20ms/step - loss: 58.5838 - output_1_loss: 0.3123 - output_2_loss: 40.2767 - output_3_loss: 17.9948 - output_1_accuracy: 0.0000e+00 - output_2_accuracy: 0.3350 - output_3_accuracy: 0.4092 - val_loss: 29.9325 - val_output_1_loss: 0.3309 - val_output_2_loss: 17.4962 - val_output_3_loss: 12.1054 - val_output_1_accuracy: 0.0000e+00 - val_output_2_accuracy: 0.5700 - val_output_3_accuracy: 0.2600
Epoch 2/10
38/38 [==============================] - 0s 9ms/step - loss: 44.0900 - output_1_loss: 0.3170 - output_2_loss: 25.5812 - output_3_loss: 18.1918 - output_1_accuracy: 0.0000e+00 - output_2_accuracy: 0.2700 - output_3_accuracy: 0.3442 - val_loss: 47.8721 - val_output_1_loss: 0.3309 - val_output_2_loss: 30.2706 - val_output_3_loss: 17.2705 - val_output_1_accuracy: 0.0000e+00 - val_output_2_accuracy: 0.5700 - val_output_3_accuracy: 0.2600
Epoch 3/10
38/38 [==============================] - 0s 9ms/step - loss: 54.1009 - output_1_loss: 0.3170